In [1]:
import tensorflow_datasets as tfds


In [2]:
#load file imbd review from the tfds library.
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

#From the list of train, take 
single_example = list(imdb['train'].take(1))[0]

#the list contain 2 kinf of tuples that is  review which is [0] and label[1]

print(single_example[1])

# 1 show positive review and 0 indicated negative review

tf.Tensor(0, shape=(), dtype=int64)


In [3]:
#contains 25000 respective sentences and labels as tensors
train_data, test_data = imdb['train'],imdb['test']

In [13]:
#Use map function to split the review from label and store in 2 different variable
train_reviews = train_data.map(lambda review, label: review)
train_labels = train_data.map(lambda review,label: label)

In [14]:
#Use map function to split the review from label and store in 2 different variable
test_reviews = test_data.map(lambda review, label: review)
test_label = test_data.map(lambda review, label: label)

In [15]:
import tensorflow as tf

In [16]:
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=10000)

vectorize_layer.adapt(train_reviews)

In [17]:
#Padding the train review 

def padding_func(sequences):
    sequences = sequences.ragged_batch(batch_size= sequences.cardinality())
    sequences = sequences.get_single_element()

    padded_sequences = tf.keras.utils.pad_sequences(sequences.numpy(), maxlen=120, truncating='post', padding='pre')
    padded_sequences = tf.data.Dataset.from_tensor_slices(padded_sequences)

    return padded_sequences

In [18]:
train_sequences = train_reviews.map(lambda text:vectorize_layer(text)).apply(padding_func)
test_sequences = test_reviews.map(lambda text:vectorize_layer(text)).apply(padding_func)


In [ ]:
#Combining sequence and labels using zip method

train_dataset_vectorized = tf.data.Dataset.zip(train_sequences, train_labels)
test_dataset_vectorized = tf.data.Dataset.zip(test_sequences, test_label)

In [21]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE
BATCH_SIZE = 32

train_dataset_final = (train_dataset_vectorized.cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       .batch(BATCH_SIZE))

test_dataset_final = (test_dataset_vectorized.cache()
                      .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       .batch(BATCH_SIZE))

In [44]:
vocab_size = 10000
embedding_dim = 16

In [45]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(120,)),
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [46]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 120, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1920)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 6)              │        11,526 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,533 (670.05 KB)

 Trainable params: 171,533 (670.05 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim),
#     tf.keras.layers.GlobalAveragePooling1D(),
#     tf.keras.layers.Dense(6,activation='relu'),
#     tf.keras.layers.Dense(1,activation='sigmoid')
# ])

In [ ]:
# vocab_size = 120
# embedding_dim = 16

In [ ]:
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Training The dataset

In [48]:
num_epoch = 10
model.fit(train_dataset_final,
          epochs = num_epoch,
          validation_data = test_dataset_final)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6374 - loss: 0.6046 - val_accuracy: 0.8179 - val_loss: 0.3925
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8793 - loss: 0.2868 - val_accuracy: 0.8168 - val_loss: 0.4193
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9555 - loss: 0.1374 - val_accuracy: 0.8080 - val_loss: 0.5111
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9929 - loss: 0.0360 - val_accuracy: 0.8019 - val_loss: 0.6086
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9992 - loss: 0.0078 - val_accuracy: 0.8044 - val_loss: 0.6808
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.8053 - val_loss: 0.7430
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 1.0000 - loss: 9.5563e-04 - val_accuracy: 0.8058 - val_loss: 0.7872
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 1.0000 - loss: 5.0114e-04 - val_accu

In [49]:
# Demonstrate the embedding

embedding_layer = model.layers[0]
embedding_weights = embedding_layer.get_weights()[0]
print(embedding_weights.shape)  #shape of (vocab_size, embedding_dim)

(10000, 16)


In [ ]:
#In order to plot it, we need to take token back into the word when look 
# at them at visualization

import io

out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')


vocabulary = vectorize_layer.get_vocabulary()

for word_num in range(1, len(vocabulary)):
    word_name =vocabulary[word_num]
    word_embedding = embedding_weights[word_num]
    out_m.write(word_name + "\n")
    out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

out_v.close()
out_m.close()    

In [52]:
import os
print(os.getcwd())

d:\Coursera\NLP with tensorflow


In [53]:
# Go to projector.tensorflow.org and upload the file vecs.tsv and meta.tsv 
# in load section. Click on sphereize data for binary look.